Importing

In [5]:
import warnings
# import cmle
import random
import numpy as np
import pandas as pd
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



Preprocessing

In [6]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

X_train = train_df.drop(columns=["Survived"])
y_train = train_df["Survived"]

numeric_features = ["Age", "Pclass", "SibSp", "Parch", "Fare"]
categorical_features = ["Embarked", "Cabin", "Ticket"]
binary_features = ["Sex"]
# drop name and passengerid

num_tran = StandardScaler()
cat_tran = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore",sparse_output=False)
)
bin_tran = OneHotEncoder(drop="if_binary",sparse_output=False)

preprocessor = make_column_transformer(
    (num_tran, numeric_features),
    (cat_tran, categorical_features),
    (bin_tran, binary_features),
    remainder = "drop"
)

X_train_transformed = preprocessor.fit_transform(X_train)

Dummy Results

In [7]:
base_pipe = make_pipeline(
    preprocessor,
    DummyClassifier(strategy="most_frequent")
)
base_pipe.fit(X_train, y_train)
predictions = base_pipe.predict(test_df)

submission_df = test_df[["PassengerId"]].copy()
submission_df["Survived"] = predictions
submission_df.to_csv("submissions/deliverable_2/dummy_submission.csv", index=False)